In [ ]:
from pathlib import Path

import azureml
from IPython.display import display, Markdown
from azureml.core import Run, Model
from azureml.core import Datastore, Experiment, ScriptRunConfig, Workspace, RunConfiguration
from azureml.core.dataset import Dataset
from azureml.data import OutputFileDatasetConfig
from azureml.core.environment import Environment
from azureml.core.runconfig import DockerConfiguration
from azureml.exceptions import UserErrorException

from model_drift import settings, helpers
# check core SDK version number
print("Azure ML SDK Version: ", azureml.core.VERSION)


In [ ]:
# Connect to workspace
ws = Workspace.from_config(settings.AZUREML_CONFIG)


In [ ]:
import re

dbg = True
log_refresh_rate = 1

# Name experiement
input_dataset_name="padchest"
experiment_name = 'vae-padchest'
if dbg:
    experiment_name += '-dbg'
model_name = 'vae-padchest-trained-frosty_dinner_h4n3bv0k-049.ckpt'
model_name = 'vae-padchest-trained-lime_night_wlbcf6py-049.ckpt'
datastore_name = "results"
env_name = "vae"

# model = Model(ws, model_name)

#Experiment
exp = Experiment(workspace=ws, name=experiment_name)

#Environment
environment_file = settings.CONDA_ENVIRONMENT_FILE
project_dir = settings.SRC_DIR
pytorch_env = Environment.from_conda_specification(env_name, file_path =str(environment_file))
pytorch_env.register(workspace=ws)
build = pytorch_env.build(workspace=ws)
pytorch_env.environment_variables["AZUREML_COMPUTE_USE_COMMON_RUNTIME"] = "false"
pytorch_env.environment_variables["RSLEX_DIRECT_VOLUME_MOUNT"] = "True"

# Run Configuration
run_config = RunConfiguration()
run_config.environment_variables["AZUREML_COMPUTE_USE_COMMON_RUNTIME"] = "false"
run_config.environment_variables["RSLEX_DIRECT_VOLUME_MOUNT"] = "True"
run_config.environment = pytorch_env
run_config.docker = DockerConfiguration(use_docker=True, shm_size="100G")

# Input Dataset
dataset = Dataset.get_by_name(ws, name=input_dataset_name)
datastore = ws.datastores[datastore_name]

output_dataset_name = re.sub(r'\W+', "", f"{model_name}".replace('-', "_"))

output_dir = f"model_outputs/{output_dataset_name}"

if dbg:
    output_dir = f"model_outputs/dbg/{output_dataset_name}"
    output_dataset_name = f"{output_dataset_name}_dbg"

output_dir = output_dir.replace("//", "/")
output_dataset = OutputFileDatasetConfig(name=output_dataset_name, destination=(datastore, output_dir.strip("/")+"/"))

args = {
'run_azure': 1,
 'model': model_name,
 'data_folder': dataset.as_named_input('dataset').as_mount(),
 'progress_bar_refresh_rate': 25 if not dbg else 1,
 'log_every_n_steps': 25 if not dbg else 1,
 'flush_logs_every_n_steps': 25 if not dbg else 1,
 'batch_size': 32,
 'accelerator': 'ddp',
 'output_dir': output_dataset.as_mount(),
 'add_name_to_output': int(dbg),
 'write_recon': 1
 }


if dbg:
    args.update({
        'limit_predict_batches': 10,
    })

for param, value in args.items():
    print(f" {param}: {value}")

print(f"Environment: {pytorch_env.name}")
print(f"Experiment: {exp.name}")

In [ ]:
compute_target = "nc24-uswest2"

config = ScriptRunConfig(
    source_directory=str(project_dir),
    script="scripts/vae/score.py",
    arguments=helpers.argsdict2list(args),
)
run_config.target = compute_target
config.run_config = run_config

run = exp.submit(config)
display(Markdown(f"""
- Experiment: [{run.experiment.name}]({run.experiment.get_portal_url()})
- Run: [{run.display_name}]({run.get_portal_url()})
- Target: {config.run_config.target}
"""))


In [ ]:
import re

dbg = True
log_refresh_rate = 1

# Name experiement
input_dataset_name="padchest"
experiment_name = 'chexpert-vae-OnPadChest-dbg'
model_name =  "chexpert-vae-wheat_book_mmkzkspg-049"
datastore_name = "vaeresults_padchest"
env_name = "vae"
compute_target = "nc6-uswest2"
compute_target = "NC24rs-v3-usw2-d"


exp = Experiment(workspace=ws, name=experiment_name)
environment_file = settings.CONDA_ENVIRONMENT_FILE
project_dir = Path("./experiment")
pytorch_env = Environment.from_conda_specification(env_name, file_path =str(environment_file))

pytorch_env.register(workspace=ws)
build = pytorch_env.build(workspace=ws)
pytorch_env.environment_variables["RSLEX_DIRECT_VOLUME_MOUNT"] = "True"
pc_dataset = Dataset.get_by_name(ws, name=input_dataset_name)

datastore = ws.datastores[datastore_name]
output_dataset_name = re.sub(r'\W+', "" ,f"{model_name}".replace('-', "_"))
output_dir =  f"{output_dataset_name}"

if dbg:
    output_dataset_name = f"{output_dataset_name}_dbg"
    output_dir = f"{output_dir}/dbg"

output_dir = output_dir.replace("//", "/")
output_dataset = OutputFileDatasetConfig(name=output_dataset_name, destination=(datastore, output_dir.strip("/")+"/"))

print("Output Dataset Name:", output_dataset_name)
print("Experiment:", exp.name)
print("Environment:", pytorch_env.name)

run_config = RunConfiguration()
run_config.environment = pytorch_env
run_config.docker = DockerConfiguration(use_docker=True, shm_size="100G")

run_config.output_data = {"output_dataset_name": output_dataset}


args = [
    "--run_azure", 1,
    "--model", model_name,
    '--data_folder', pc_dataset.as_named_input('padchestv1').as_mount(),
    '--progress_bar_refresh_rate', log_refresh_rate,
    "--log_every_n_steps", log_refresh_rate,
    "--flush_logs_every_n_steps", log_refresh_rate,
    "--batch_size", 128,
    "--accelerator", "ddp",
    "--output_dir", output_dataset.as_mount(),
    "--write_recon", 1,
    "--write_grid",  1,
    "--latent_output_dir", "./outputs/",
    "--append_run_name", 1,
    ]

if dbg:
    args += [
        '--limit_predict_batches', 10,
    ]


config = ScriptRunConfig(
    source_directory = str(project_dir), 
    script = "score.py",
    arguments=args,
)
run_config.target = compute_target
config.run_config = run_config

run = exp.submit(config)
display(Markdown(f"""
- Experiement: [{run.experiment.name}]({run.experiment.get_portal_url()})
- Run: [{run.display_name}]({run.get_portal_url()})
- Target: {config.run_config.target}
"""))

In [ ]:
# TODO: Move to another notebook or delete?

# experiment_name = "dataset_debug"
# chex_dataset = Dataset.get_by_name(ws, name='chexpert')
# pc_dataset = Dataset.get_by_name(ws, name=input_dataset_name)


# exp = Experiment(workspace=ws, name=experiment_name)
# environment_file = settings.CONDA_ENVIRONMENT_FILE
# project_dir = Path("./experiment")
# pytorch_env = Environment.from_conda_specification(env_name, file_path =str(environment_file))

# pytorch_env.register(workspace=ws)
# build = pytorch_env.build(workspace=ws)

# pytorch_env.environment_variables["RSLEX_DIRECT_VOLUME_MOUNT"] = "True"

# print("Experiment:", exp.name)
# print("Environment:", pytorch_env.name)

# run_config = RunConfiguration()
# run_config.environment = pytorch_env
# run_config.docker = DockerConfiguration(use_docker=True, shm_size="100G")

# args = [
#     '--data1', pc_dataset.as_named_input('padchestv1').as_mount(),
#     '--data2', chex_dataset.as_named_input('chexpertv1').as_mount(),
#     ]


# config = ScriptRunConfig(
#     source_directory = str(project_dir), 
#     script = "dataset_test.py",
#     arguments=args,
# )

# config.run_config = run_config

# config.run_config.target = compute_target

# # config.run_config.target = "nc6-uswest2"

# run = exp.submit(config)
# display(Markdown(f"""
# - Experiement: [{run.experiment.name}]({run.experiment.get_portal_url()})
# - Run: [{run.display_name}]({run.get_portal_url()})
# - Target: {config.run_config.target}
# """))